## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Afsin,TR,38.25,36.91,82.15,23,55,14.29,broken clouds
1,1,Thompson,CA,55.74,-97.86,46.56,93,100,11.50,mist
2,2,Esperance,AU,-33.87,121.90,55.78,83,52,12.44,broken clouds
3,3,Tuchola,PL,53.59,17.86,57.00,52,100,8.75,overcast clouds
4,4,Tuatapere,NZ,-46.13,167.68,48.79,93,100,2.71,overcast clouds


In [3]:
# Ask the user for desired min and max temp
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [29]:
# Filter city_data_df to isolate places that meet user input
pref_dest_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]

In [30]:
# Determine if there are any empty (NaN) rows.
# ** Interpreting empty rows as any row that is missing any amount of data. There are no fully empty rows in the
# data set that I have found thus far **
null_rows = pref_dest_df.isnull().sum().sum()

2

In [33]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if null_rows > 0:
    clean_df = pref_dest_df.dropna(axis=0,how='any')
else:
    clean_df = pref_dest_df.copy()

In [32]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Esperance,AU,55.78,broken clouds,-33.87,121.90,
3,Tuchola,PL,57.00,overcast clouds,53.59,17.86,
9,Richards Bay,ZA,63.05,light rain,-28.78,32.04,
13,Kaeo,NZ,55.33,overcast clouds,-35.10,173.78,
15,Aberdeen,GB,67.66,overcast clouds,57.14,-2.10,
16,Hobart,AU,55.26,clear sky,-42.88,147.33,
23,Mazyr,BY,57.33,overcast clouds,52.05,29.25,
27,Swan River,CA,53.49,overcast clouds,52.11,-101.27,
30,Albany,US,66.67,overcast clouds,42.60,-73.97,
31,Codrington,AU,55.20,light rain,-38.27,141.97,


In [34]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found -> skipping")    

Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping
Hotel not found -> skipping


In [58]:
# Drop the rows where there is no Hotel Name.
# This method was chosen over a search using iterrows to preserve data types. Documentation also discourages changing
# values in a DataFrame during iterrows.

# Set rows with empty hotel names to NaN
hotel_df.loc[hotel_df["Hotel Name"] == ""] = np.nan

# Count num rows with NaN hotel names
no_hotel = hotel_df["Hotel Name"].isnull().sum().sum()

# Remove rows if needed
if no_hotel > 0:
    clean_hotel_df = hotel_df.dropna(axis=0,how='any')
else:
    clean_hotel_df = hotel_df.copy()

In [59]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [74]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.

# ** The instructions did not specify that hotel name should be added, however it is displayed in the screenshot of what
# the map should look like, therefore I have added it.

info_box_template = """
<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Current Description</dt><dd>{Current Description}</dd>

<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [75]:
# Add a marker layer for each city to the map.
fig = gmaps.figure(center=(20,5),zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))